In [ ]:
"""
MONOLITH QAE BENCH — Quickstart

What you need to provide:
-------------------------------------
1) CIRCUIT TEMPLATE (Cell 3)
   - Implement your full QAE as two callables inside the class:
       self._encoder_block(params_per_layer)
       self._decoder_block(params_per_layer)
     Each should apply RX/RY/RZ + entanglers (or your custom brick).
   - Choose how to use your single parameter array:
       decoder_mode = "adjoint_encoder"  -> decoder = adjoint(encoder)
       decoder_mode = "mirror_same"      -> decoder uses the SAME angles, forward
       decoder_mode = "separate_split"   -> split array into [encoder | decoder]
     (Set this in Cell 4 via JSON or keep default; also auto-inferred from length.)

2) MODEL PATHS (Cell 7)
   - Put your .json files into MODEL_PATHS = {"Tag": "/path/to/model.json", ...}
   - Standard filename pattern is supported (auto meta):
       4q_2l_2t_3ls_01.json  -> nq=4, latent=2, trash=2, layers=3, instance=01
     Otherwise we look for "architecture" in your JSON.

3) Run cells 1 → 7.

Standardized evaluation:
------------------------
• Deterministic Mackey–Glass datasets (MG_A, MG_B), window sizes {4,6}.
• Fixed noisy variants at σ=0.10 (change EVAL_SIGMA if you must).
• Metrics: ΔMSE% per-window mean + 95% CI, sign test, success rate.
• Visuals: recon examples, robustness (kept but σ list defaults to 0.10),
  full-series, panelized views, “error lens”, and heatmaps (if many models).

Notes:
------
• If your JSON has a single flat array, we infer n_layers from len(params)/(n_qubits*3).
  If the length equals 2*n_layers*n_qubits*3 we switch to "separate_split".
• Latent/trash: set n_latent and trash_wires (defaults to [latent..nq-1]).
• Deterministic seeds → identical fixed noisy sets across runs.
"""

In [1]:
import os, re, json, math, hashlib, warnings
from dataclasses import dataclass
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as pnp

np.set_printoptions(suppress=True, precision=6)
plt.rcParams["figure.figsize"] = (6.5, 4)

# ---------- scaling & angles ----------
def values_to_v01(v, low, high):
    v = np.asarray(v, dtype=float)
    return (v - low) / max(1e-12, (high - low))

def v01_to_values(v01, low, high):
    v01 = np.asarray(v01, dtype=float)
    return low + v01 * (high - low)

def readoutZ_to_values(z, low, high):
    # RY(pi*v01) → z = cos(pi*v01)  ⇒ v01 = arccos(z)/pi
    z = np.clip(np.asarray(z, dtype=float), -0.999999, 0.999999)
    v01 = np.arccos(z) / np.pi
    return v01_to_values(v01, low, high)

# ---------- stats ----------
def bootstrap_ci_mean(x, B=3000, alpha=0.05, rng=None):
    x = np.asarray(x, dtype=float)
    if x.size == 0: return (np.nan, np.nan, np.nan)
    rng = np.random.default_rng(None if rng is None else rng)
    n = x.size
    xb = np.empty(B, float)
    for b in range(B):
        idx = rng.integers(0, n, size=n)
        xb[b] = np.mean(x[idx])
    xb.sort()
    lo = xb[int((alpha/2)*B)]
    hi = xb[int((1 - alpha/2)*B) - 1]
    return float(np.mean(x)), float(lo), float(hi)

def sign_test_pvalue(diffs):
    """Two-sided exact sign test on paired diffs (>0 counts as win)."""
    diffs = np.asarray(diffs, dtype=float)
    wins  = int(np.sum(diffs > 0))
    losses= int(np.sum(diffs < 0))
    n     = wins + losses
    if n == 0: return 1.0
    tail = sum(math.comb(n, k) for k in range(0, min(wins, losses)+1)) / (2**n)
    return float(min(1.0, 2*tail))

def symmetric_delta_pct(noisy_mse, deno_mse, floor=1e-12):
    """2*(noisy - deno) / max(noisy + deno, floor) → symmetric, bounded."""
    n = float(noisy_mse); d = float(deno_mse)
    denom = max(n + d, floor)
    return 200.0 * (n - d) / denom

In [2]:
# Global evaluation sigma (kept everywhere)
EVAL_SIGMA = 0.10
NOISE_GRID = (EVAL_SIGMA,)  # build datasets only for this sigma

def mackey_glass(length=1300, tau=17, beta=0.2, gamma=0.1, n=10, x0=1.2, dt=1.0):
    x = np.zeros(length + tau + 1, dtype=float)
    x[:tau+1] = x0
    for t in range(tau, length + tau):
        xt = x[t]; xt_tau = x[t - tau]
        dx = beta * xt_tau / (1.0 + xt_tau**n) - gamma * xt
        x[t+1] = xt + dx * dt
    return x[tau+1:]

def scale_to_range(y, low=0.2, high=0.8):
    y = np.asarray(y, dtype=float)
    ymin, ymax = float(y.min()), float(y.max())
    if ymax - ymin < 1e-12:
        return np.full_like(y, (low+high)/2), (low, high)
    z = (y - ymin) / (ymax - ymin)
    return low + z * (high - low), (low, high)

def make_windows(ts, size, step):
    ts = np.asarray(ts, dtype=float)
    return np.array([ts[i:i+size] for i in range(0, len(ts)-size+1, step)], dtype=float)

def _stable_seed(tag: str) -> int:
    h = hashlib.sha256(tag.encode("utf-8")).digest()
    return int.from_bytes(h[:8], "little") & 0x7FFFFFFF

def add_gaussian_noise_series(series, sigma, low, high, seed):
    rng = np.random.default_rng(seed)
    noise = rng.normal(0.0, sigma * (high - low), size=series.shape)
    x = series + noise
    return np.clip(x, low, high)

@dataclass
class Dataset:
    name: str
    clean: np.ndarray
    scale_low: float
    scale_high: float
    windows_clean: np.ndarray
    split_idx: int
    noisy_series_by_sigma: dict
    noisy_windows_by_sigma: dict

def build_standard_datasets(window_sizes=(4,6), window_step=1, noise_grid=NOISE_GRID):
    mgA = mackey_glass(length=1300, tau=17)
    mgB = mackey_glass(length=1300, tau=30)

    out = {}
    for W in window_sizes:
        sA, (loA, hiA) = scale_to_range(mgA, 0.2, 0.8)
        sB, (loB, hiB) = scale_to_range(mgB, 0.2, 0.8)
        winA = make_windows(sA, W, window_step)
        winB = make_windows(sB, W, window_step)
        splitA = int(0.75 * len(winA))
        splitB = int(0.75 * len(winB))

        def noisy_maps(name, scaled, low, high):
            noisy_series, noisy_windows = {}, {}
            for sigma in noise_grid:
                seed = _stable_seed(f"{name}|W={W}|sigma={sigma:.3f}|v1")
                ns = add_gaussian_noise_series(scaled, sigma, low, high, seed=seed)
                noisy_series[round(sigma,3)] = ns
                noisy_windows[round(sigma,3)] = make_windows(ns, W, window_step)
            return noisy_series, noisy_windows

        nsA_s, nsA_w = noisy_maps("MG_A", sA, loA, hiA)
        nsB_s, nsB_w = noisy_maps("MG_B", sB, loB, hiB)
        out.setdefault(W, {})
        out[W]["MG_A"] = Dataset("MG_A", sA, loA, hiA, winA, splitA, nsA_s, nsA_w)
        out[W]["MG_B"] = Dataset("MG_B", sB, loB, hiB, winB, splitB, nsB_s, nsB_w)

    print("Datasets ready for window sizes:", list(out.keys()))
    return out

# Build both sizes so nq=4 and nq=6 models are covered
WINDOW_STEP = 1
DATASETS_BY_W = build_standard_datasets(window_sizes=(4,6), window_step=WINDOW_STEP, noise_grid=NOISE_GRID)

Datasets ready for window sizes: [4, 6]


In [3]:
class MonolithFullQAE:
    """
    Monolithic full-QAE with a SINGLE parameter array.
    decoder_mode:
      - "adjoint_encoder"  -> decoder = adjoint(encoder) using same angles
      - "mirror_same"      -> decoder uses the same brick forward (not adjoint)
      - "separate_split"   -> params = [encoder | decoder] (equal halves)
    """
    def __init__(self,
                 n_qubits: int,
                 n_layers: int,
                 n_latent: int,
                 trash_wires: list | None,
                 scale_low: float,
                 scale_high: float,
                 params: np.ndarray,       # flat array from JSON
                 decoder_mode: str = "adjoint_encoder"):
        self.n_qubits = int(n_qubits)
        self.n_layers = int(n_layers)
        self.n_latent = int(n_latent)
        self.latent_wires = list(range(self.n_latent))
        self.trash_wires = list(trash_wires) if trash_wires is not None else list(range(self.n_latent, self.n_qubits))
        self.scale_low  = float(scale_low)
        self.scale_high = float(scale_high)
        self.decoder_mode = str(decoder_mode)

        fp = np.asarray(params, dtype=float).ravel()
        self.full_params = pnp.asarray(fp, requires_grad=False)

        per_layer = self.n_qubits * 3  # RX, RY, RZ per qubit
        if self.decoder_mode == "separate_split":
            expected = 2 * self.n_layers * per_layer
            if fp.size != expected:
                raise ValueError(f"[separate_split] Expect len(params) == 2*L*Q*3 = {expected}, got {fp.size}")
            half = fp.size // 2
            self.enc_flat = pnp.asarray(fp[:half], requires_grad=False)
            self.dec_flat = pnp.asarray(fp[half:],  requires_grad=False)
        else:
            need = self.n_layers * per_layer
            if fp.size < need:
                raise ValueError(f"Expect len(params) >= L*Q*3 = {need}, got {fp.size}")
            self.enc_flat = pnp.asarray(fp[:need], requires_grad=False)
            self.dec_flat = self.enc_flat  # same angles

        self.dev = qml.device("default.qubit", wires=self.n_qubits)
        self._q_forward = None
        self._q_latents = None
        self._q_trash   = None

    # ======== TODO: If your ansatz differs, edit these three helpers ==========
    def _layer_rxryrz_ring(self, params_layer):
        """Default brick: RX/RY/RZ per qubit + ring CNOT entanglers."""
        P = np.asarray(params_layer, dtype=float).reshape(self.n_qubits, 3)
        for q in range(self.n_qubits):
            rx, ry, rz = P[q]
            qml.RX(rx, wires=q); qml.RY(ry, wires=q); qml.RZ(rz, wires=q)
        for q in range(self.n_qubits - 1):
            qml.CNOT(wires=[q, q+1])
        qml.CNOT(wires=[self.n_qubits - 1, 0])

    def _encoder_block(self, flat_params):
        P = pnp.asarray(flat_params).reshape(self.n_layers, self.n_qubits, 3)
        for l in range(self.n_layers):
            self._layer_rxryrz_ring(P[l])

    def _decoder_block(self, flat_params):
        if self.decoder_mode == "adjoint_encoder":
            qml.adjoint(self._encoder_block)(flat_params)
        else:
            P = pnp.asarray(flat_params).reshape(self.n_layers, self.n_qubits, 3)
            for l in range(self.n_layers):
                self._layer_rxryrz_ring(P[l])
    # ==========================================================================

    def _embed_from_values(self, values_window):
        v01 = values_to_v01(values_window, self.scale_low, self.scale_high)
        thetas = np.pi * v01
        for i, th in enumerate(thetas):
            qml.RY(float(th), wires=i)

    def map_expZ_to_values(self, z_all):
        return readoutZ_to_values(z_all, self.scale_low, self.scale_high)

    # QNodes -------------------------------------------------------------------
    def _build_forward(self):
        @qml.qnode(self.dev, diff_method=None)
        def qnode(values_window):
            self._embed_from_values(values_window)
            self._encoder_block(self.enc_flat)
            self._decoder_block(self.dec_flat)
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
        return qnode

    def _build_latents_after_encoder(self):
        @qml.qnode(self.dev, diff_method=None)
        def qnode(values_window):
            self._embed_from_values(values_window)
            self._encoder_block(self.enc_flat)
            return [qml.expval(qml.PauliZ(i)) for i in self.latent_wires]
        return qnode

    def _build_trash_probs_after_encoder(self):
        @qml.qnode(self.dev, diff_method=None)
        def qnode(values_window):
            self._embed_from_values(values_window)
            self._encoder_block(self.enc_flat)
            return qml.probs(wires=self.trash_wires)
        return qnode

    # Public inference ---------------------------------------------------------
    def forward_expZ(self, values_window):
        if self._q_forward is None: self._q_forward = self._build_forward()
        return np.asarray(self._q_forward(values_window), dtype=float)

    def latents_after_encoder_expZ(self, values_window):
        if self._q_latents is None: self._q_latents = self._build_latents_after_encoder()
        return np.asarray(self._q_latents(values_window), dtype=float)

    def trash_probs_after_encoder(self, values_window):
        if self._q_trash is None: self._q_trash = self._build_trash_probs_after_encoder()
        return np.asarray(self._q_trash(values_window), dtype=float)

    def predict_values(self, values_window):
        return self.map_expZ_to_values(self.forward_expZ(values_window))

    def describe(self):
        print(f"Monolith QAE — nq={self.n_qubits}, layers={self.n_layers}, "
              f"latent={self.n_latent}, trash={len(self.trash_wires)} ({self.trash_wires}), "
              f"decoder_mode={self.decoder_mode}, scale=[{self.scale_low},{self.scale_high}]")

In [4]:
def _sha256_params(obj):
    buf = bytearray()
    def walk(x):
        if isinstance(x, (list, tuple)):
            for y in x: walk(y)
        elif isinstance(x, np.ndarray):
            buf.extend(np.ascontiguousarray(x, dtype=np.float64).tobytes())
        elif isinstance(x, (float, int, np.floating, np.integer)):
            buf.extend(np.asarray([x], dtype=np.float64).tobytes())
        else:
            buf.extend(str(x).encode("utf-8"))
    walk(obj)
    return hashlib.sha256(buf).hexdigest()

# Standard filename:  4q_2l_2t_3ls_01.json
_STD_RE = re.compile(r"(?P<nq>\d+)q_(?P<nl>\d+)l_(?P<nt>\d+)t_(?P<L>\d+)ls_(?P<inst>\d+)\.json$", re.IGNORECASE)

def parse_std_filename(path):
    m = _STD_RE.search(os.path.basename(path))
    if not m: return None
    d = {k:int(v) for k,v in m.groupdict().items()}
    return {"n_qubits": d["nq"], "n_latent": d["nl"], "n_trash": d["nt"], "n_layers": d["L"], "instance": d["inst"]}

@dataclass
class ModelEntry:
    name: str
    path: str
    n_qubits: int
    n_latent: int
    trash_wires: list
    scale_low: float
    scale_high: float
    noise_sigma_train: float
    meta: dict
    params: dict
    fingerprint: str
    impl: object

# --------- JSON parser for "one-array" monolithic bundles --------------------
def parse_model_json_monolith(path, default_decoder_mode="adjoint_encoder"):
    with open(path, "r", encoding="utf-8") as f:
        J = json.load(f)

    # Locate a single parameter array
    full_params = None
    if isinstance(J.get("parameters"), dict):
        for k, v in J["parameters"].items():
            if isinstance(v, (list, tuple)) and len(v) > 0 and np.isscalar(v[0]):
                full_params = np.asarray(v, dtype=float).ravel()
                break
    if full_params is None:
        for key in ["full_params","theta","weights","params","phi","psi"]:
            if key in J and isinstance(J[key], (list, tuple)):
                full_params = np.asarray(J[key], dtype=float).ravel()
                break
    if full_params is None:
        raise ValueError("Could not find a single flat parameter array in JSON.")

    arch = J.get("architecture", {})
    ds   = J.get("dataset", {})
    run  = J.get("run", {})

    # filename fallback
    fn_meta = parse_std_filename(path) or {}

    n_qubits = int(arch.get("n_qubits", ds.get("window_size", fn_meta.get("n_qubits", 4))))
    n_latent = int(arch.get("n_latent", fn_meta.get("n_latent", max(1, n_qubits//2))))
    n_trash  = int(arch.get("n_trash",  fn_meta.get("n_trash", n_qubits - n_latent)))
    trash_wires = arch.get("trash_wires", list(range(n_latent, n_qubits)))
    scale_low  = float(ds.get("scale_low", 0.2))
    scale_high = float(ds.get("scale_high", 0.8))

    # infer n_layers and decoder_mode from length unless explicitly given
    per_layer = n_qubits * 3
    n_layers = arch.get("n_layers")
    decoder_mode = arch.get("decoder_mode", default_decoder_mode)

    if n_layers is None:
        L1 = len(full_params) / per_layer
        L2 = len(full_params) / (2*per_layer)
        if abs(L1 - round(L1)) < 1e-9:
            n_layers = int(round(L1))
            # keep decoder_mode (default or JSON override)
        elif abs(L2 - round(L2)) < 1e-9:
            n_layers = int(round(L2))
            decoder_mode = "separate_split"
        else:
            raise ValueError(f"Cannot infer n_layers from len(params)={len(full_params)} and n_qubits={n_qubits}.")
    else:
        n_layers = int(n_layers)
        if len(full_params) == 2 * n_layers * per_layer:
            decoder_mode = "separate_split"

    info = {
        "n_qubits": n_qubits, "n_layers": n_layers,
        "n_latent": n_latent, "trash_wires": list(trash_wires),
        "scale_low": scale_low, "scale_high": scale_high,
        "n_trash": n_trash,
        "decoder_mode": decoder_mode,
        "noise_sigma_train": float(run.get("sigma_train", EVAL_SIGMA)),
        "timestamp": J.get("timestamp"),
        "source_schema": "monolith_one_array",
        **arch
    }
    return {"full_params": full_params, "info": info}

def instantiate_model_monolith(parsed):
    i = parsed["info"]
    impl = MonolithFullQAE(
        n_qubits=i["n_qubits"], n_layers=i["n_layers"],
        n_latent=i["n_latent"], trash_wires=i["trash_wires"],
        scale_low=i["scale_low"], scale_high=i["scale_high"],
        params=parsed["full_params"],                      
        decoder_mode=i.get("decoder_mode","adjoint_encoder"),
    )
    return impl

def load_models_monolith(model_paths: dict):
    registry = {}
    for name, path in model_paths.items():
        try:
            P = parse_model_json_monolith(path)
            impl = instantiate_model_monolith(P)
            fp = _sha256_params([P["full_params"]])
            i = P["info"]
            entry = ModelEntry(
                name=name, path=path, n_qubits=i["n_qubits"], n_latent=i["n_latent"],
                trash_wires=list(i["trash_wires"]), scale_low=i["scale_low"], scale_high=i["scale_high"],
                noise_sigma_train=float(i["noise_sigma_train"]), meta=i,
                params={"full_params": P["full_params"]},
                fingerprint=fp, impl=impl
            )
            registry[name] = entry
            print(f"✓ Loaded {name}  (nq={entry.n_qubits}, L={i['n_layers']}, "
                  f"latent={entry.n_latent}, trash={len(entry.trash_wires)}, "
                  f"mode={i.get('decoder_mode')}, σ_train={entry.noise_sigma_train:g})")
            impl.describe()
        except Exception as e:
            print(f"✗ {name}: {e}")
    return registry

print("Monolith loader ready.")

Monolith loader ready.


In [5]:
# Keep σ fixed everywhere and test N windows per dataset
N_EVAL_WINDOWS = 20

# Sanity: datasets-by-window must exist and contain σ=0.10
for w, dsdict in DATASETS_BY_W.items():
    for dsname, ds in dsdict.items():
        assert round(EVAL_SIGMA,3) in ds.noisy_windows_by_sigma, (
            f"{dsname} (w={w}) missing σ={EVAL_SIGMA:.3f}. Rebuild datasets with NOISE_GRID=(0.10,)."
        )

@dataclass
class EvalResult:
    noisy_mse: np.ndarray
    model_mse: np.ndarray
    delta_pct: np.ndarray
    delta_pct_sym: np.ndarray
    recon_values: np.ndarray
    noisy_values: np.ndarray
    clean_values: np.ndarray
    lat_clean: np.ndarray|None
    lat_noisy: np.ndarray|None
    p00_trash: np.ndarray|None

def _sigma_key(s: float) -> float:
    return float(np.round(float(s), 3))

def _pick_sigma(entry, override):
    # "model" → use entry.noise_sigma_train; else explicit float
    return float(entry.noise_sigma_train) if (override == "model") else float(override)

def eval_model_on_dataset(entry, ds, n_eval=20, sigma_override=EVAL_SIGMA):
    """Deterministic eval on the dataset's fixed noisy windows."""
    s = _pick_sigma(entry, sigma_override) if isinstance(sigma_override, str) else float(sigma_override)
    s_key = _sigma_key(s)
    if s_key not in ds.noisy_windows_by_sigma:
        avail = ", ".join([f"{k:.3f}" for k in sorted(ds.noisy_windows_by_sigma)])
        raise ValueError(f"{ds.name}: σ={s_key:.3f} not in [{avail}]")

    cleanW = ds.windows_clean[ds.split_idx:]
    noisyW = ds.noisy_windows_by_sigma[s_key][ds.split_idx:]
    N = min(int(n_eval), len(cleanW))
    cleanW = cleanW[:N]; noisyW = noisyW[:N]

    impl = entry.impl
    recon, nmse, dmse, d_pct, d_sym = [], [], [], [], []
    lat_c, lat_n, p00 = [], [], []

    for w_clean, w_noisy in zip(cleanW, noisyW):
        z = impl.forward_expZ(w_noisy)
        y = impl.map_expZ_to_values(z)
        recon.append(y)

        mse_n = float(np.mean((w_clean - w_noisy)**2))
        mse_d = float(np.mean((w_clean - y      )**2))
        nmse.append(mse_n); dmse.append(mse_d)
        d_pct.append(0.0 if mse_n < 1e-12 else 100.0 * (mse_n - mse_d) / mse_n)
        d_sym.append(200.0 * (mse_n - mse_d) / max(mse_n + mse_d, 1e-12))

        # Optional hooks (monolith might not implement; safe in try/except)
        try:
            lc = impl.latents_after_encoder_expZ(w_clean)
            ln = impl.latents_after_encoder_expZ(w_noisy)
            lat_c.append(lc); lat_n.append(ln)
        except Exception:
            lat_c = lat_n = None
        try:
            P = impl.trash_probs_after_encoder(w_noisy)
            p00.append(float(P[0]))
        except Exception:
            p00 = None

    return EvalResult(
        noisy_mse=np.asarray(nmse), model_mse=np.asarray(dmse),
        delta_pct=np.asarray(d_pct), delta_pct_sym=np.asarray(d_sym),
        recon_values=np.asarray(recon), noisy_values=noisyW, clean_values=cleanW,
        lat_clean=None if lat_c is None else np.asarray(lat_c),
        lat_noisy=None if lat_n is None else np.asarray(lat_n),
        p00_trash=None if p00 is None else np.asarray(p00)
    )

def summarize_eval(er: EvalResult):
    mean_imp, lo_imp, hi_imp = bootstrap_ci_mean(er.delta_pct)
    mean_sym, lo_sym, hi_sym = bootstrap_ci_mean(er.delta_pct_sym)
    p_val = sign_test_pvalue(er.noisy_mse - er.model_mse)
    succ  = 100.0 * float(np.mean(er.delta_pct > 0))
    return {
        "noisy_MSE_mean": float(np.mean(er.noisy_mse)),
        "model_MSE_mean": float(np.mean(er.model_mse)),
        "delta_pct_mean": float(mean_imp),
        "delta_pct_CI95": [float(lo_imp), float(hi_imp)],
        "delta_pct_sym_mean": float(mean_sym),
        "delta_pct_sym_CI95": [float(lo_sym), float(hi_sym)],
        "sign_test_p": float(p_val),
        "success_rate_pct": float(succ),
        "n_windows": int(er.noisy_mse.size)
    }

In [6]:
import pandas as pd

# ---------- small-window visuals ----------
def plot_reconstruction_example(er: EvalResult, idx=0, title="Reconstruction"):
    c, n, d = er.clean_values[idx], er.noisy_values[idx], er.recon_values[idx]
    xs = np.arange(len(c))
    plt.figure()
    plt.plot(xs, c, label="clean"); plt.plot(xs, n, label="noisy"); plt.plot(xs, d, label="denoised")
    plt.title(title); plt.xlabel("t"); plt.ylabel("value"); plt.legend(); plt.show()

def plot_delta_distributions(er_by_model, title="ΔMSE% (per-window)"):
    plt.figure()
    for name, er in er_by_model.items():
        plt.hist(er.delta_pct, bins=12, alpha=0.5, label=name)
    plt.title(title); plt.xlabel("improvement %"); plt.ylabel("count"); plt.legend(); plt.show()

# ---------- robustness (σ list allowed but defaults to 0.10 only) ----------
def plot_robustness(entry, ds, sigmas=(0.10,), n_eval=N_EVAL_WINDOWS):
    xs, means, lows, highs = [], [], [], []
    available = set(ds.noisy_windows_by_sigma.keys())
    for s in sigmas:
        s_key = float(np.round(float(s), 3))
        if s_key not in available:
            print(f"[robustness] skip σ={s_key:.3f} (dataset has {sorted(available)})")
            continue
        er = eval_model_on_dataset(entry, ds, n_eval=n_eval, sigma_override=float(s))
        m, lo, hi = bootstrap_ci_mean(er.delta_pct)
        xs.append(float(s)); means.append(m); lows.append(lo); highs.append(hi)
    if not xs:
        print("[robustness] nothing to plot."); return
    plt.figure(); plt.plot(xs, means, marker="o"); plt.fill_between(xs, lows, highs, alpha=0.2)
    plt.xlabel("σ"); plt.ylabel("ΔMSE% (mean, 95%CI)"); plt.title(f"Robustness on {ds.name} — {entry.name}"); plt.show()

# ---------- tables & heatmaps ----------
def summaries_to_df(all_summaries, registry):
    rows=[]
    for dsname, bymodel in all_summaries.items():
        for name, sm in bymodel.items():
            e = registry[name]
            layers = e.meta.get("n_layers", getattr(e.impl, "n_layers", None))
            rows.append({
                "dataset": dsname, "model": name,
                "n_qubits": e.n_qubits, "n_latent": e.n_latent, "n_trash": len(e.trash_wires),
                "layers": layers,
                "delta_pct_mean": sm["delta_pct_mean"],
                "delta_pct_lo": sm["delta_pct_CI95"][0],
                "delta_pct_hi": sm["delta_pct_CI95"][1],
                "success_rate_pct": sm["success_rate_pct"],
                "p": sm["sign_test_p"],
            })
    return pd.DataFrame(rows)

def heatmap_by_arch(df, dataset, value_col="delta_pct_mean"):
    sub = df[df["dataset"]==dataset]
    if sub.empty:
        print(f"[heatmap] no rows for {dataset}"); return
    for (nl, nt), group in sub.groupby(["n_latent","n_trash"]):
        piv = group.pivot_table(index="n_qubits", columns="layers", values=value_col, aggfunc="mean")
        plt.figure(figsize=(5.2,3.6)); plt.imshow(piv.values, aspect="auto")
        plt.xticks(range(piv.shape[1]), piv.columns); plt.yticks(range(piv.shape[0]), piv.index)
        plt.colorbar(label=value_col)
        plt.title(f"{dataset} — mean {value_col} (latent={nl}, trash={nt})")
        plt.xlabel("layers"); plt.ylabel("n_qubits"); plt.tight_layout(); plt.show()

# ---------- full-series tools ----------
def _flatten_avg(windows, step):
    windows = np.asarray(windows, dtype=float)
    N, W = windows.shape; L = (N-1)*step + W
    acc = np.zeros(L); cnt = np.zeros(L)
    for i in range(N):
        s=i*step; e=s+W; acc[s:e]+=windows[i]; cnt[s:e]+=1
    return acc/np.maximum(cnt,1e-12)

def reconstruct_full_series(entry, ds, sigma=EVAL_SIGMA, step=1):
    s_key = float(np.round(float(sigma), 3))
    if s_key not in ds.noisy_windows_by_sigma:
        avail = ", ".join([f"{k:.3f}" for k in sorted(ds.noisy_windows_by_sigma)])
        raise ValueError(f"{ds.name}: σ={s_key:.3f} not in [{avail}]")
    cleanW = np.asarray(ds.windows_clean, dtype=float)
    noisyW = np.asarray(ds.noisy_windows_by_sigma[s_key], dtype=float)
    preds  = np.asarray([entry.impl.map_expZ_to_values(entry.impl.forward_expZ(w)) for w in noisyW])
    flat_c = _flatten_avg(cleanW, step); flat_n = _flatten_avg(noisyW, step); flat_d = _flatten_avg(preds, step)
    mse_n=float(np.mean((flat_c-flat_n)**2)); mse_d=float(np.mean((flat_c-flat_d)**2))
    d_pct=0.0 if mse_n<1e-12 else 100.0*(mse_n-mse_d)/mse_n
    return {"clean":flat_c,"noisy":flat_n,"deno":flat_d,"mse_noisy":mse_n,"mse_deno":mse_d,"delta_pct":d_pct,"sigma":s_key}

def plot_full_series(rec, title):
    L=len(rec["clean"]); xs=np.arange(L)
    plt.figure(figsize=(9.8,4.0))
    plt.plot(xs, rec["clean"], label="clean")
    plt.plot(xs, rec["noisy"], label=f"noisy (MSE={rec['mse_noisy']:.5f})")
    plt.plot(xs, rec["deno"],  label=f"deno (MSE={rec['mse_deno']:.5f}, Δ%={rec['delta_pct']:+.1f})")
    plt.xlabel("t"); plt.ylabel("value"); plt.title(title); plt.legend(); plt.tight_layout(); plt.show()

def plot_full_series_panels(entry, ds, sigma=EVAL_SIGMA, panel_len=180, ncols=3, nrows=2, start=0, sharey=True):
    rec = reconstruct_full_series(entry, ds, sigma=sigma)
    c,n,d = rec["clean"], rec["noisy"], rec["deno"]; L=len(c)
    fig,axes = plt.subplots(nrows, ncols, figsize=(4.5*ncols,2.75*nrows), sharey=sharey)
    axes=np.ravel(axes)
    for k,ax in enumerate(axes):
        s = start + k*panel_len; e=min(s+panel_len, L)
        if s>=L: ax.axis("off"); continue
        xs=np.arange(s,e); cc=c[s:e]; nn=n[s:e]; dd=d[s:e]
        mse_n=float(np.mean((cc-nn)**2)); mse_d=float(np.mean((cc-dd)**2))
        d_pct=0.0 if mse_n<1e-12 else 100.0*(mse_n-mse_d)/mse_n
        ax.plot(xs,cc,label="clean"); ax.plot(xs,nn,label=f"noisy (MSE={mse_n:.5f})")
        ax.plot(xs,dd,label=f"deno (MSE={mse_d:.5f}, Δ%={d_pct:+.1f})")
        if k==0: ax.legend(); ax.set_ylabel("value")
        ax.set_title(f"{s}–{e}"); ax.set_xlabel("t")
    fig.suptitle(f"{entry.name} — {ds.name} (σ={rec['sigma']:.3f}), panels of {panel_len}", y=1.02)
    plt.tight_layout(); plt.show()

def error_lens(entry, ds, sigma=EVAL_SIGMA, seg=150, start=0, length=None):
    rec = reconstruct_full_series(entry, ds, sigma=sigma)
    c,n,d = rec["clean"], rec["noisy"], rec["deno"]; L=len(c)
    if length is None: length=L-start
    s,e=int(start),int(min(start+length,L))
    se_n=(n-c)**2; se_d=(d-c)**2; imp=se_n-se_d
    mean_n=float(se_n.mean()); mean_d=float(se_d.mean())
    d_pct=100.0*(mean_n-mean_d)/max(mean_n,1e-12); frac=100.0*float(np.mean(imp>0))
    print(f"{ds.name} | {entry.name} | σ={rec['sigma']:.3f}\n"
          f"Global MSE: noisy={mean_n:.5f} deno={mean_d:.5f} Δ%={d_pct:+.1f}\n"
          f"Samples improved: {frac:.1f}%")
    xs=np.arange(s,e)
    plt.figure(figsize=(10,3.1)); plt.plot(xs, se_n[s:e], label="(clean-noisy)^2")
    plt.plot(xs, se_d[s:e], label="(clean-deno)^2"); plt.title(f"Per-sample squared errors — {ds.name} [{s}:{e}]")
    plt.xlabel("t"); plt.ylabel("squared error"); plt.legend(); plt.tight_layout(); plt.show()

    plt.figure(figsize=(10,2.5)); imp_seg=imp[s:e]; plt.plot(xs, imp_seg, lw=0.8)
    plt.fill_between(xs,0,imp_seg, where=(imp_seg>=0), alpha=0.35)
    plt.fill_between(xs,0,imp_seg, where=(imp_seg<0),  alpha=0.25)
    plt.axhline(0,color="k",lw=0.8); plt.title(f"Improvement per sample — {ds.name} [{s}:{e}]")
    plt.xlabel("t"); plt.ylabel("ΔSE"); plt.tight_layout(); plt.show()

    bins=list(range(0,L,seg)); contrib=[imp[i:i+seg].sum() for i in bins]; centers=[i+seg/2 for i in bins]
    plt.figure(figsize=(10,3.0)); plt.bar(centers, contrib, width=0.8*seg); plt.axhline(0,color="k",lw=0.8)
    plt.title(f"Contribution to total improvement by {seg}-sample segments — {ds.name}")
    plt.xlabel("segment center"); plt.ylabel("Σ ΔSE"); plt.tight_layout(); plt.show()

In [7]:
# Put your monolith JSONs here (standard names help later aggregation).
MODEL_PATHS = {
    # "Mentor_4q_2l_2t_3ls_01": "/abs/or/rel/path/4q_2l_2t_3ls_01.json",
    # "Mentor_6q_4l_2t_2ls_07": "/abs/or/rel/path/6q_4l_2t_2ls_07.json",

    # (optional) keep Jacob’s old bundle for side-by-side comparisons:
    # "Me_run_20250824_nq4_s010": "/path/to/Jacob/.../model_bundle.json",
}

REG = load_models_monolith(MODEL_PATHS)

def run_benchmark(model_registry, n_eval=N_EVAL_WINDOWS, sigma_override=EVAL_SIGMA):
    all_results, all_summaries = {}, {}
    for name, entry in model_registry.items():
        w = entry.n_qubits
        if w not in DATASETS_BY_W:
            warnings.warn(f"No dataset for window_size={w}; skipping {name}."); continue
        for dsname, ds in DATASETS_BY_W[w].items():
            er = eval_model_on_dataset(entry, ds, n_eval=n_eval, sigma_override=sigma_override)
            sm = summarize_eval(er)
            all_results.setdefault(dsname, {})[name]  = er
            all_summaries.setdefault(dsname, {})[name]= sm
            print(f"{dsname} | {name:24s} Δ%={sm['delta_pct_mean']:+5.1f} "
                  f"(CI {sm['delta_pct_CI95'][0]:+.1f},{sm['delta_pct_CI95'][1]:+.1f})  "
                  f"succ={sm['success_rate_pct']:4.1f}%  p={sm['sign_test_p']:.4f}")
    return all_results, all_summaries

ALL_RESULTS, ALL_SUMMARIES = run_benchmark(REG, n_eval=N_EVAL_WINDOWS, sigma_override=EVAL_SIGMA)

# Quick table
if ALL_SUMMARIES:
    rows=[]
    for dsname, d in ALL_SUMMARIES.items():
        for name, sm in d.items():
            e = REG[name]
            rows.append({
                "dataset": dsname, "model": name,
                "n_qubits": e.n_qubits, "n_latent": e.n_latent, "n_trash": len(e.trash_wires),
                "layers": e.meta.get("n_layers", getattr(e.impl, "n_layers", None)),
                "delta_pct_mean": sm["delta_pct_mean"],
                "delta_pct_lo": sm["delta_pct_CI95"][0],
                "delta_pct_hi": sm["delta_pct_CI95"][1],
                "success_rate_pct": sm["success_rate_pct"],
                "p": sm["sign_test_p"],
            })
    df = pd.DataFrame(rows).sort_values(["dataset","n_qubits","layers","model"])
    print("\n== Summary (head) =="); print(df.head(12).to_string(index=False))

# Quick visuals for the first loaded model
if REG:
    name0, entry0 = next(iter(REG.items()))
    nq = entry0.n_qubits
    assert nq in DATASETS_BY_W, f"Missing datasets for window size {nq}"
    dsA = DATASETS_BY_W[nq]["MG_A"]; dsB = DATASETS_BY_W[nq]["MG_B"]

    erA = ALL_RESULTS["MG_A"][name0]; erB = ALL_RESULTS["MG_B"][name0]
    plot_reconstruction_example(erA, idx=0, title=f"{name0} — MG_A (σ={EVAL_SIGMA:g})")
    plot_reconstruction_example(erB, idx=0, title=f"{name0} — MG_B (σ={EVAL_SIGMA:g})")

    # full-series
    recA = reconstruct_full_series(entry0, dsA, sigma=EVAL_SIGMA); plot_full_series(recA, f"{name0} — MG_A (σ={recA['sigma']:.3f})")
    recB = reconstruct_full_series(entry0, dsB, sigma=EVAL_SIGMA); plot_full_series(recB, f"{name0} — MG_B (σ={recB['sigma']:.3f})")

    # panels + error lens on MG_B
    plot_full_series_panels(entry0, dsB, sigma=EVAL_SIGMA, panel_len=180, ncols=3, nrows=2, start=0)
    error_lens(entry0, dsB, sigma=EVAL_SIGMA, seg=150, start=0,   length=900)
    error_lens(entry0, dsB, sigma=EVAL_SIGMA, seg=150, start=900, length=400)